In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import auc,precision_recall_curve,roc_curve,confusion_matrix
import os,sys
import pickle
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import random
import seaborn as sns
np.random.seed(10)
random.seed(10)


## Functions for loading data

In [25]:
def load_training_and_validataion_dataset(path_to_partitions,train_splits):
    import random
    training_partions = random.sample(range(10),train_splits)
    # training_partions = [9, 0, 6, 3, 4, 8, 1, 7]
    validation_partions = [i for i in range(10) if i not in training_partions]

    # path_to_partitions = "../data/partitions"
    partitions = []
    for file in os.listdir(path_to_partitions):
        path_to_file = os.path.join(path_to_partitions,file)
        data = pd.read_csv(path_to_file,sep="\t",names=["peptide","label","HLA"])
        partitions.append(data)
    training_df = pd.concat([partitions[i] for i in training_partions])
    validation_df = pd.concat([partitions[i] for i in validation_partions])
    return training_df, validation_df,training_partions,validation_partions

def retrieve_information_from_df(data_split,entire_df):
    potential = []
    immunogenicity = []
    tested = []
    responded = []
    for i,row in data_split.iterrows():
        peptide, HLA = row["peptide"], row['HLA']
        original_entry = entire_df[(entire_df['peptide']==peptide) & (entire_df['HLA'] == HLA)]
        assert len(original_entry) == 1
        potential.append(float(original_entry['potential']))
        immunogenicity.append(original_entry['immunogenicity'].values[0])
        tested.append(int(original_entry['test']))
        responded.append(int(original_entry['respond']))
     
    data_split['potential'] = potential
    data_split['immunogenicity'] = immunogenicity
    data_split['test'] = tested
    data_split['respond'] = responded

    return data_split  


def encode_aaindex(aa_seq,aaindex_PCA):
    aa_seq = list(aa_seq)
    encoded_aa_seq = []
    for aa in aa_seq:
        print(aaindex_PCA.loc[aa])
        sys.exit(1)
    ...

## Loading the data

In [26]:
# Loading the databases
aaindex_PCA = pd.read_csv('../data/PCA_repr_aa.csv',index_col=0)
hla_database = pd.read_csv('../data/formatted_MHC_pseudo.dat', sep=' ',index_col=0)
hla_dic = hla_database.to_dict("dict")["pseudo"]
# Load dataset
entire_df = pd.read_csv('../data/filtered_data_IEDB_4_tested_len_9_10_full_HLA_IFNg_assay.csv')

entire_df = pd.read_csv('../data/deep_immuno_2.csv')
# Allocating the partitions of the trainign and validation data
training_df, validation_df,training_partions,validation_partions = load_training_and_validataion_dataset(path_to_partitions="../data/deepimmuno_parts",train_splits=8)

# Creating the training dataframe (With correct information such as tested and positive subjects aswell as label)
training_df_entire = retrieve_information_from_df(training_df,entire_df)
# Shuffling the dataframe
training_df_entire = training_df_entire.sample(frac=1, random_state=1).reset_index(drop=True)

# Creating the validation dataframe (With correct information such as tested and positive subjects aswell as label)
validation_df_entire = retrieve_information_from_df(validation_df,entire_df)
# Shuffling the dataframe
validation_df_entire = validation_df_entire.sample(frac=1, random_state=1).reset_index(drop=True)

# Encode data


encode_aaindex("AKTQDE",aaindex_PCA)
sys.exit(1)

training_dataset_encoded = construct_aaindex(training_df_entire, hla_dic, after_pca)
peptide_train = pull_peptide_aaindex(training_dataset_encoded)
HLA_train = pull_hla_aaindex(training_dataset_encoded)
label_train = pull_label_aaindex(training_dataset_encoded)


val_dataset_encoded = construct_aaindex(validation_df_entire, hla_dic, after_pca)
peptide_val = pull_peptide_aaindex(val_dataset_encoded)
HLA_val = pull_hla_aaindex(val_dataset_encoded)
label_val = pull_label_aaindex(val_dataset_encoded)

peptide_train = peptide_train.reshape(-1,1,10,12).astype('float32')
HLA_train = HLA_train.reshape(-1,1,46,12).astype('float32')
label_train = label_train.astype('float32')

peptide_val = peptide_val.reshape(-1,1,10,12).astype('float32')
HLA_val = HLA_val.reshape(-1,1,46,12).astype('float32')
label_val = label_val.astype('float32')



PC1      0.240664
PC2      5.659694
PC3     16.599203
PC4      1.309795
PC5      3.358757
PC6      1.957431
PC7     -3.656903
PC8     -0.662709
PC9      3.544847
PC10    -2.280156
PC11     0.825385
PC12     1.695619
Name: A, dtype: float64


SystemExit: 1

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
